   # Within-fire variability 
   
   Workflow to extract pixels from imagery and boostrap samples to get median and mean for each image.

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
import dask
%matplotlib inline
%tb


In [28]:
# required functions
%run processing_functions.py
%run ../pixel_to_df.py

## Test workflow on single fire

What is the distribution of pixel counts in  defol/non-defoliated between and within fires?


Goals: 
1) create reference distribution from one of defoliation / non-defoiated halves
2) from the opposite half, subsample to match sample size of reference. 
3) boostrap resample to get CI and distrbution of values. 
4) compare distribution

Steps:
1) get coordinates and return them as a dataframe with x and y
2) add column to df which says whether it is defoliated or non-defoliated (binary)
3) extract pixel values? 
4) randomly select a subsample of pixels from df, get values...etc.?

* turn into 4d array with time, band, x, y - band would be nbr and defoliated (1,0)


NOTES:
- values to be standardized when in DF format after pixel extraction

In [48]:
# read in data and get coordinates
ds = rx.open_rasterio("/Users/jgoldman/Desktop/chp3/clipped/nbr/non-defoliated/COC16_1997_1038.nc", masked=True)

lat = ds.coords['y'].values
lon = ds.coords['x'].values


411

In [59]:
# another option

time, lat2, lon2 = ds.indexes.values()
time
ds.count()

ds2= ds.isel(time=2)
df = ds2.to_dataframe()
df

time  crs  nbr
y         x                           
49.943680 -80.599497  1998.0    0  NaN
          -80.599228  1998.0    0  NaN
          -80.598958  1998.0    0  NaN
          -80.598689  1998.0    0  NaN
          -80.598419  1998.0    0  NaN
...                      ...  ...  ...
49.853669 -80.490083  1998.0    0  NaN
          -80.489813  1998.0    0  NaN
          -80.489544  1998.0    0  NaN
          -80.489274  1998.0    0  NaN
          -80.489005  1998.0    0  NaN

[137685 rows x 3 columns]

In [79]:
# random sample for a given year.
# for each sample take the coordinates and get values across df.


def random_sample(da,
                  n, 
                  dim_slice,
                  keep_cols=True):
    """
    take a data array, turn to df and get random sample
    da = xarray data array with n dimensions
    n = number of samples
    dim_slice = slice along certain time dimension

    """
    # slice along dimension
    da2= da.isel(time=dim_slice)
    #to dataframe
    df = ds2.to_dataframe()

    # sample location
    samples = []
    no_of_points = n
    # random sample of n points
    sample_loc = df.dropna().sample(n=int(round(no_of_points)))
    samples.append(sample_loc)
    #join back into single datafame
    all_samples = pd.concat([samples[i] for i in range(0,len(samples))])

   
    
    # x and y to lat and lon column
    all_samples.reset_index(inplace = True)
    all_samples = all_samples.rename(columns={'y': 'Lat', 'x': 'Long'}) 
    
    if keep_cols==False:
        #remove crs, nbr and time cols
        all_samples = all_samples.drop(columns =['crs', 'time', 'nbr'])
    
    return(all_samples)

In [82]:
df = random_sample(ds, 1000, 1, False)
df

,Lat,Long
0,49.875228,-80.579555
1,49.907567,-80.535627
2,49.866874,-80.545059
3,49.918078,-80.551527
4,49.905681,-80.525117
...,...,...
995,49.878462,-80.565541
996,49.886547,-80.559343
997,49.873072,-80.556917
998,49.862831,-80.544520


In [ ]:
# merge the following workflow with random sample function

pixel_values = img.sel(x=lat[i], y=lon[i], method="nearest")
pixel_values_df = pixel_values.reset_coords(drop=True).to_dataframe(name="nbr")
df = std_nbr(pixel_values_df)